In [1]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from tqdm import tqdm
import nltk

# Ensure nltk is installed and download necessary resources
import os
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import nltk
    nltk.download('punkt')
except ImportError:
    install('nltk')
    import nltk
    nltk.download('punkt')

print("NLTK Done, onto Generator")

# Generator and Discriminator Models
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(self.model.config.hidden_size, self.model.config.vocab_size)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.linear(outputs.last_hidden_state)
        return logits
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

print("LOADING DATASETS")
# Load dataset and split into training and validation sets
dataset = load_dataset('wmt14', 'fr-en', split='train')
val_dataset = load_dataset('wmt14', 'fr-en', split='validation')

small_dataset = dataset.shuffle(seed=42).select(range(25000))
small_val_dataset = val_dataset.shuffle(seed=42).select(range(1000))

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print("STARTED PREPROCESSING")
def preprocess(examples):
    inputs = tokenizer(examples['translation']['en'], return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    targets = tokenizer(examples['translation']['fr'], return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    return {
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze(),
        'target_ids': targets['input_ids'].squeeze()
    }

# Use multiprocessing to speed up the preprocessing
print("train preprocess")
train_dataset = small_dataset.map(preprocess, remove_columns=['translation'], num_proc=6)
print("val preprocess")
val_dataset = small_val_dataset.map(preprocess, remove_columns=['translation'], num_proc=6)

# Define your dataset class
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {key: torch.tensor(val) for key, val in item.items()}

# Data collation function for efficient batching
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    target_ids = torch.stack([item['target_ids'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target_ids': target_ids}

# Create DataLoaders
train_custom_dataset = CustomDataset(train_dataset)
val_custom_dataset = CustomDataset(val_dataset)
train_dataloader = DataLoader(train_custom_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
validation_dataloader = DataLoader(val_custom_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Initialize models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = Adam(generator.parameters(), lr=0.0002)
optimizer_D = Adam(discriminator.parameters(), lr=0.0004)

# Loss functions
adversarial_loss = nn.BCEWithLogitsLoss().to(device)
translation_loss = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id).to(device)

# Gradient clipping
max_grad_norm = 1.0

# Mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Training loop
num_epochs = 25
for epoch in tqdm(range(1, num_epochs + 1)):
    generator.train()
    discriminator.train()
    
    for batch_idx, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        with torch.cuda.amp.autocast():
            real_labels = torch.ones(input_ids.size(0), 1).to(device) * 0.9  # label smoothing
            real_output = discriminator(input_ids, attention_mask)
            d_loss_real = adversarial_loss(real_output, real_labels)
            
            fake_logits = generator(input_ids, attention_mask)
            fake_translations = torch.argmax(fake_logits, dim=-1)
            fake_labels = torch.zeros(input_ids.size(0), 1).to(device)
            fake_output = discriminator(fake_translations.detach(), attention_mask)
            d_loss_fake = adversarial_loss(fake_output, fake_labels)
            
            d_loss = (d_loss_real + d_loss_fake) / 2

        scaler.scale(d_loss).backward()
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_grad_norm)
        scaler.step(optimizer_D)
        scaler.update()

        # Train Generator
        optimizer_G.zero_grad()
        with torch.cuda.amp.autocast():
            translated_logits = generator(input_ids, attention_mask)
            g_loss_adv = adversarial_loss(discriminator(torch.argmax(translated_logits, dim=-1), attention_mask), real_labels)
            g_loss_trans = translation_loss(translated_logits.view(-1, translated_logits.size(-1)), target_ids.view(-1))
            g_loss = g_loss_adv + g_loss_trans

        scaler.scale(g_loss).backward()
        torch.nn.utils.clip_grad_norm_(generator.parameters(), max_grad_norm)
        scaler.step(optimizer_G)
        scaler.update()

        # Print batch losses
        if batch_idx % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(train_dataloader)}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}")

    # Print epoch summary after each epoch
    print(f"Epoch [{epoch}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}")


# Save the trained models
torch.save(generator.state_dict(), 'generator_model.pth')
torch.save(discriminator.state_dict(), 'discriminator_model.pth')

# Translation and BLEU computation functions
def translate_texts(generator, dataloader):
    generator.eval()
    all_translations = []
    all_references = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_ids = batch['target_ids'].to(device)

            translations = generator(input_ids, attention_mask)
            translations = translations.argmax(dim=-1).cpu().numpy()

            for i in range(len(translations)):
                decoded_translation = tokenizer.decode(translations[i], skip_special_tokens=True)
                decoded_reference = tokenizer.decode(target_ids[i].cpu().numpy(), skip_special_tokens=True)

                all_translations.append(decoded_translation)
                all_references.append([decoded_reference])

    return all_translations, all_references

# Load generator model
generator.load_state_dict(torch.load('generator_model.pth'))

# After training
validation_translations, validation_references = translate_texts(generator, validation_dataloader)

# Compute BLEU score using nltk
def compute_bleu_nltk(references, translations):
    references = [[ref.split()] for ref in references]
    translations = [trans.split() for trans in translations]
    bleu_score = nltk.translate.bleu_score.corpus_bleu(references, translations)
    return bleu_score

validation_bleu = compute_bleu_nltk([ref[0] for ref in validation_references], validation_translations)
print(f"Validation BLEU score: {validation_bleu * 100:.2f}")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
NLTK Done, onto Generator
LOADING DATASETS


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/40836715 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

...

 96%|█████████▌| 24/25 [8:18:35<20:46, 1246.86s/it]

Epoch [24/25], d_loss: 0.6838, g_loss: 4.4485
Epoch [25/25], Batch [0/782], d_loss: 0.6938, g_loss: 4.5828
Epoch [25/25], Batch [10/782], d_loss: 0.6918, g_loss: 4.4385
Epoch [25/25], Batch [20/782], d_loss: 0.6924, g_loss: 4.5706
Epoch [25/25], Batch [30/782], d_loss: 0.6942, g_loss: 4.2816
Epoch [25/25], Batch [40/782], d_loss: 0.6924, g_loss: 4.4249
Epoch [25/25], Batch [50/782], d_loss: 0.6827, g_loss: 4.8922
Epoch [25/25], Batch [60/782], d_loss: 0.6873, g_loss: 4.4385
Epoch [25/25], Batch [70/782], d_loss: 0.6866, g_loss: 4.5854
Epoch [25/25], Batch [80/782], d_loss: 0.6915, g_loss: 4.5478
Epoch [25/25], Batch [90/782], d_loss: 0.6800, g_loss: 4.6453
Epoch [25/25], Batch [100/782], d_loss: 0.6905, g_loss: 4.6734
Epoch [25/25], Batch [110/782], d_loss: 0.6864, g_loss: 4.2387
Epoch [25/25], Batch [120/782], d_loss: 0.6910, g_loss: 4.4242
Epoch [25/25], Batch [130/782], d_loss: 0.6865, g_loss: 4.6170
Epoch [25/25], Batch [140/782], d_loss: 0.6888, g_loss: 4.5703
Epoch [25/25], Batch

100%|██████████| 25/25 [8:39:22<00:00, 1246.48s/it]

Epoch [25/25], d_loss: 0.6920, g_loss: 4.5105


Validation BLEU score: 0.41
